## 最適化問題
$$ \min_h \|\bm{g}-F^\top \bm{h}\|_2^2+\lambda_1\|\bm{h}\|_1 + \lambda_2\|D\bm{h}\|_1$$

In [3]:
import random
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import norm, spsolve
from typing import Callable, Union
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import seaborn as sns
from PIL import Image
import os
import re

In [2]:
# パラメータ設定
n = 64
m = 128
LAMBDA1 = 1
LAMBDA2 = 1
SEED = 5
RATIO = 0.1
DATA_PATH = '../../OneDrive - m.titech.ac.jp/Lab/data'
IMG_NAME = 'hadamard'
DIRECTORY = DATA_PATH + '/240626'
SETTING=f"{IMG_NAME}_l1_p-{int(100*RATIO)}_lmd-{int(LAMBDA)}_seed-{SEED}"

In [ ]:
def matrix2vector(matrix):
    return matrix.T.flatten()


def vector2matrix(vector, s, t):
    return vector.reshape(s, t).T


def mult_mass(F, h, M):
    N = F.shape[1]
    h = h.reshape(N, M)
    res = F @ h
    return res.flatten()


def images_to_matrix(folder_path, convert_gray=True, rand=True, ratio=RATIO):
    files = os.listdir(folder_path)
    files.sort(key=lambda f: int(re.search(f"{IMG_NAME}_(\d+).png", f).group(1)))
    if rand:
        random.seed(SEED)
        random.shuffle(files)

    total_files = len(files)
    number_of_files_to_load = int(total_files * ratio)
    selected_files = files[:number_of_files_to_load]
    selected_files.sort(key=lambda f: int(re.search(f"{IMG_NAME}_(\d+).png", f).group(1)))

    images = []
    use_list = []

    for file in selected_files:
        index = int(re.sub(r'\D', '', file))
        use_list.append(index)
        img = Image.open(os.path.join(folder_path, file))
        if convert_gray:
            img = img.convert('L')
        img_array = np.asarray(img).flatten()
        img_array = img_array / 255
        images.append(img_array)

    return np.column_stack(images), use_list


In [ ]:
def prox_l1(x: np.ndarray, alpha: float) -> np.ndarray:
    """Proximal operator for L1 norm."""
    return np.sign(x) * np.maximum(np.abs(x) - alpha, 0)


def prox_l1_conj(x: np.ndarray, alpha: float) -> np.ndarray:
    """Proximal operator for conjugate L1 norm."""
    return x - alpha * prox_l1(x/alpha, 1/alpha)


def grad_f(h: np.ndarray, F: np.ndarray, g: np.ndarray) -> np.ndarray:
    """Gradient of f(h) = ||g - Fh||_2^2."""
    return 2 * F.T @ (F @ h - g)


def primal_dual_splitting(
    F: np.ndarray,
    D: np.ndarray,
    g: np.ndarray,
    lambda1: float,
    lambda2: float,
    max_iter: int = 1000,
    tol: float = 1e-2,
) -> tuple[np.ndarray, dict]:
    """
    Solve the optimization problem:
    min_h ||g-Fh||_2^2 + lambda1||h||_1 + lambda2||Dh||_1
    using the primal-dual splitting method.

    Args:
        F (np.ndarray): Matrix F in the problem formulation.
        D (np.ndarray): Gradient matrix D.
        g (np.ndarray): Vector g in the problem formulation.
        lambda1 (float): Regularization parameter for L1 norm of h.
        lambda2 (float): Regularization parameter for L1 norm of Dh.
        h_init (np.ndarray, optional): Initial guess for h. If None, initialized with zeros.
        y_init (np.ndarray, optional): Initial guess for y. If None, initialized with zeros.
        max_iter (int): Maximum number of iterations.
        tol (float): Tolerance for convergence.
        adaptive_params (bool): Whether to use adaptive step sizes.

    Returns:
        tuple[np.ndarray, dict]: Solution h and a dictionary containing additional information.
    """
    m, n = F.shape
    h = np.zeros(n)
    y = np.zeros(D.shape[0])

    # Compute Lipschitz constant of grad_f
    L = norm(F, 2) ** 2

    # Initialize step sizes
    tau = 1.0 / L
    sigma = 1.0 / (norm(D, 2) ** 2)

    objective_values = []
    for k in range(max_iter):
        h_old = h.copy()
        y_old = y.copy()

        # Update primal variable h
        grad = grad_f(h_old, F, g)
        h = prox_l1(h - tau * (grad - D.T @ y), tau * lambda1)

        # Update dual variable y
        y = prox_l1_conj(y + sigma * D @ (2 * h - h_old), sigma / lambda2)

        # Compute objective value
        obj_value = np.linalg.norm(
            g - F @ h)**2 + lambda1 * np.linalg.norm(h, 1) + lambda2 * np.linalg.norm(D @ h, 1)
        objective_values.append(obj_value)

        # Check convergence
        primal_residual = np.linalg.norm(
            h - h_old) / max(np.linalg.norm(h), 1e-8)
        dual_residual = np.linalg.norm(
            y - y_old) / max(np.linalg.norm(y), 1e-8)
        print(
            f"iter={k}, obj={obj_value:.4f}, primal_res={primal_residual:.4f}, dual_res={dual_residual:.4f}")
        if primal_residual < tol and dual_residual < tol:
            break

    info = {
        "iterations": k + 1,
        "objective_values": objective_values,
        "final_objective": objective_values[-1],
        "primal_residual": primal_residual,
        "dual_residual": dual_residual
    }

    return h, info

In [ ]:
# load images
G, use = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/")
F, _ = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_input/")
print("K=", F.shape[1])
white_img = Image.open(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/{IMG_NAME}_1.png")
white_img = white_img.convert('L')
white_img = np.asarray(white_img) / 255
white = white_img.flatten()
white = white[:, np.newaxis]
H1 = np.tile(white, F.shape[1])
F_hat = 2 * F - 1
G_hat = 2 * G - H1

g = matrix2vector(G_hat)

In [ ]:
D = sparse.eye(n, n, format="csc")

In [ ]:
h, info = primal_dual_splitting(F_hat, D, g, LAMBDA1, LAMBDA2, max_iter=500)